## Importação de Bibliotecas

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import random
from selenium.webdriver.common.action_chains import ActionChains
import time, random
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time, random
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException
import time, random
from dataclasses import dataclass
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    TimeoutException, ElementClickInterceptedException,
    StaleElementReferenceException
)
from selenium.webdriver.support import expected_conditions as EC
import os, json, time, random, datetime, requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import setuptools
import undetected_chromedriver as uc
from selenium.common.exceptions import WebDriverException



## Definições de Click e Variáveis Humanizadas 

Define onde o boot vai clicar, no caso escolhe de forma aleatória o pixel que ele deve clicar para executa a ação.
Define também o tempo de digitação de cada digito de forma aleatória, em vez de simplemente colar o número.

In [ ]:

def _only_digits(s: str) -> str:
    return ''.join(ch for ch in (s or '') if ch.isdigit())


@dataclass
class HumanConfig:
    per_key_min: float = 0.010
    per_key_max: float = 0.090
    micro_break_every_min: int = 3
    micro_break_every_max: int = 6
    micro_break_ms_min: float = 0.060
    micro_break_ms_max: float = 0.180
    think_ms_min: float = 0.180
    think_ms_max: float = 0.420
    mistake_prob: float = 0.00
    between_cpfs_min: float = 1.5
    between_cpfs_max: float = 4.5
    coffee_every: int = 10
    coffee_ms_min: float = 20.0
    coffee_ms_max: float = 40.0
    enable_pointer_move: bool = False

    # --- NOVO ---
    click_noise_prob: float = 0.45     # % de vezes que o clique será em ponto aleatório do elemento
    page_noise_prob: float = 0.20      # % de vezes que haverá um clique "fora" (fundo da página)
    element_edge_margin_px: int = 6    # evita clicar colado nas bordas
    prefer_center_bias: float = 0.65   # 0=uniforme; 1=favor centro (gauss)
    hover_before_click_ms_min: float = 0.05
    hover_before_click_ms_max: float = 0.20
    pointer_steps_min: int = 3         # passos para o movimento suave
    pointer_steps_max: int = 6


class HumanBehavior:
    def __init__(self, driver, wait, cfg: HumanConfig = None):
        self.driver = driver
        self.wait = wait
        self.cfg = cfg or HumanConfig()

    def _pause(self, a, b): time.sleep(random.uniform(a, b))

    def wait_active(self, el, timeout=2.0):
        end = time.time() + timeout
        while time.time() < end:
            if self.driver.execute_script("return document.activeElement===arguments[0];", el):
                return True
            time.sleep(0.05)
        return False

    def scroll_into_view_center(self, el):
        self.driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth',block:'center'});", el)
        time.sleep(0.08)

    def safe_focus_input(self, input_by, label_selector=None, max_tries=3):
        last_exc = None
        for _ in range(max_tries):
            try:
                el = self.wait.until(EC.presence_of_element_located(input_by))
                self.scroll_into_view_center(el)

                # 1) Tenta pelo label (frequente quando o label cobre o input)
                if label_selector:
                    labels = self.driver.find_elements(By.CSS_SELECTOR, label_selector)
                    if labels:
                        try:
                            labels[0].click()
                            if self.wait_active(el, timeout=1.5): return el
                        except ElementClickInterceptedException:
                            pass

                # 2) Foco via JS (não “humano”, mas seguro)
                self.driver.execute_script("arguments[0].focus();", el)
                if self.wait_active(el, timeout=1.5): return el

                # 3) JS click como fallback
                try:
                    self.driver.execute_script("arguments[0].click();", el)
                    if self.wait_active(el, timeout=1.5): return el
                except Exception:
                    pass

                time.sleep(0.15)
            except (StaleElementReferenceException, ElementClickInterceptedException) as e:
                last_exc = e
                time.sleep(0.20)
        if last_exc: raise last_exc
        raise Exception("Não foi possível focar o input.")

    def clear_like_human(self, el):
        el.send_keys(Keys.CONTROL, 'a')
        self._pause(0.04, 0.09)
        el.send_keys(Keys.BACK_SPACE)
        self._pause(0.03, 0.08)

    def type_like_human(self, el, texto: str):
        micro_every = random.randint(self.cfg.micro_break_every_min, self.cfg.micro_break_every_max)
        for i, ch in enumerate(texto):
            # (Opcional) erro e correção
            if self.cfg.mistake_prob > 0 and random.random() < self.cfg.mistake_prob:
                el.send_keys(random.choice('0123456789'))
                self._pause(self.cfg.per_key_min, self.cfg.per_key_max)
                el.send_keys(Keys.BACK_SPACE)

            el.send_keys(ch)
            self._pause(self.cfg.per_key_min, self.cfg.per_key_max)

            if i and i % micro_every == 0:
                self._pause(self.cfg.micro_break_ms_min, self.cfg.micro_break_ms_max)
                micro_every = random.randint(self.cfg.micro_break_every_min, self.cfg.micro_break_every_max)

    def wait_field_value_digits(self, el, expected_digits: str, timeout=5):
        end = time.time() + timeout
        while time.time() < end:
            val = el.get_attribute("value") or ""
            if _only_digits(val).endswith(expected_digits):
                return True
            time.sleep(0.05)
        return False

    def click_like_human(self, el):
        try:
            el.click()
        except ElementClickInterceptedException:
            # Clique via JS como fallback (não “humano”, mas resiliente)
            self.driver.execute_script("arguments[0].click();", el)

    def between_cpfs_pause(self, idx):
        # pausa “humana” entre CPFs
        self._pause(self.cfg.between_cpfs_min, self.cfg.between_cpfs_max)
        # coffee break periódico
        if self.cfg.coffee_every and (idx + 1) % self.cfg.coffee_every == 0:
            print(f"[HUMANO] Pausa maior após {idx+1} CPFs…")
            self._pause(self.cfg.coffee_ms_min, self.cfg.coffee_ms_max)

def captcha_desafio_visivel(driver) -> bool:
    try:
        return len(driver.find_elements(By.CSS_SELECTOR, "iframe[src*='api2/bframe']")) > 0
    except Exception:
        return False

def aguardar_usuario_resolver_captcha(driver, timeout=180):
    if not captcha_desafio_visivel(driver): return True
    print(f"[CAPTCHA] Desafio detectado. Resolva na janela do Chrome (timeout {timeout}s).")
    try: print('\a', end='')  # beep opcional
    except Exception: pass
    end = time.time() + timeout
    while time.time() < end:
        if not captcha_desafio_visivel(driver):
            print("[CAPTCHA] Resolvido. Seguindo.")
            return True
        time.sleep(0.5)
    print("[CAPTCHA] Timeout aguardando solução.")
    return False


def _rect(self, el):
    # Usa WebElement.rect e faz fallback via JS se precisar
    r = el.rect
    if r and r.get('width') and r.get('height'):
        return r
    return self.driver.execute_script("""
        const b = arguments[0].getBoundingClientRect();
        return {x:b.left, y:b.top, width:b.width, height:b.height};
    """, el)
def _rand_point_in(self, el):
    r = self._rect(el)
    m = self.cfg.element_edge_margin_px
    w = max(0, r['width'] - 2*m)
    h = max(0, r['height'] - 2*m)
    # viés para o centro (gauss) ou uniforme
    def _biased(dim):
        if random.random() < self.cfg.prefer_center_bias:
            # gauss centrada no meio
            val = random.gauss(mu=dim/2, sigma=dim/6)  # ~99.7% dentro
            return min(max(0, val), dim)
        else:
            return random.uniform(0, dim)
    x = m + _biased(w)
    y = m + _biased(h)
    # offsets relativos ao canto SUP-ESQ do elemento (API Selenium)
    return (x, y)
def _smooth_move_to_offset(self, el, xoffset, yoffset):
    """Movimenta o mouse em pequenos passos até o offset dentro do elemento."""
    steps = random.randint(self.cfg.pointer_steps_min, self.cfg.pointer_steps_max)
    # Ponto de partida: outro ponto aleatório do mesmo elemento (não o alvo final)
    sx, sy = self._rand_point_in(el)
    actions = ActionChains(self.driver)
    # Vai até um ponto de "aproximação"
    actions.move_to_element_with_offset(el, int(sx), int(sy)).pause(random.uniform(0.02, 0.08))
    # Caminho em passos até o destino (xoffset, yoffset)
    dx = (xoffset - sx) / steps
    dy = (yoffset - sy) / steps
    for i in range(steps):
        # jitter pequeno para "mão humana"
        jx = random.uniform(-0.8, 0.8)
        jy = random.uniform(-0.8, 0.8)
        actions.move_by_offset(int(dx + jx), int(dy + jy)).pause(random.uniform(0.015, 0.06))
    actions.perform()
def click_like_human(self, el):
    """Clique que pode ocorrer em diferentes partes do elemento, com movimento suave."""
    try:
        # garante visibilidade
        self.scroll_into_view_center(el)
        # decide se usa um ponto aleatório ou o centro
        if random.random() < self.cfg.click_noise_prob:
            x, y = self._rand_point_in(el)
        else:
            # centro aproximado
            r = self._rect(el)
            x, y = r['width']/2, r['height']/2
        if self.cfg.enable_pointer_move:
            self._smooth_move_to_offset(el, x, y)
            time.sleep(random.uniform(self.cfg.hover_before_click_ms_min,
                                      self.cfg.hover_before_click_ms_max))
            ActionChains(self.driver).click().perform()
        else:
            # sem mover ponteiro: usa API com offset direto
            ActionChains(self.driver).move_to_element_with_offset(el, int(x), int(y)).click().perform()
    except ElementClickInterceptedException:
        # fallback robusto
        self.driver.execute_script("arguments[0].click();", el)
def random_page_click_noise(self):
    """Às vezes clica em uma área 'vazia' da página (fundo/body)."""
    if random.random() >= self.cfg.page_noise_prob:
        return
    try:
        body = self.driver.find_element(By.TAG_NAME, "body")
        # escolhe um offset seguro dentro do body
        r = self._rect(body)
        margin = 20
        px = random.randint(margin, max(margin+1, int(r['width'] - margin)))
        py = random.randint(int(r['height']*0.65), max(int(r['height']*0.95), int(r['height']*0.80)))
        # move e clica
        ActionChains(self.driver).move_to_element_with_offset(body, px, py).pause(random.uniform(0.02, 0.08)).click().perform()
        time.sleep(random.uniform(0.05, 0.18))
    except Exception:
        pass  # ruído é opcional; ignora se não der


### Inicialização do Navegador

In [ ]:
# Iniciar o driver
driver = webdriver.Chrome()
driver.maximize_window() # Maximizar a janela pode ajudar a evitar problemas com elementos responsivos
wait = WebDriverWait(driver, 20)

# Acessar a página de login
driver.get('https://servicos.niointernet.com.br/ajuda/servicos/segunda-via/resultado ')
time.sleep(3)

In [ ]:
# Lista para armazenar os resultados
resultados = []

# URL do portal de negociação
url_negociacao = 'https://servicos.niointernet.com.br/ajuda/servicos/segunda-via/resultado '

### Leitura Excel

In [ ]:
df = pd.read_excel(r'C:\Users\luigi.faria\Desktop\Divessos\Oi_negocia\teste.xlsx')

if 'Status da Dívida' not in df.columns:
    df['Status da Dívida'] = ''  # cria a coluna onde vamos escrever os resultados

def set_result(idx, msg):
    """Escreve o resultado na linha do CPF e faz um log."""
    df.at[idx, 'Status da Dívida'] = msg
    print(f"[PLANILHA] Linha {idx+2} -> {msg}")  # +2: cabeçalho + índice base 0

df

### Formatação de CPF para 11 digitos

In [ ]:
def formatar_cpf(cpf):
    """Garante que o CPF tenha 11 dígitos, adicionando zeros à esquerda."""
    return str(cpf).zfill(11)

### Variáveis de Humanização

In [ ]:

hb = HumanBehavior(driver, wait, HumanConfig(
    per_key_min=0.012,
    per_key_max=0.070,
    think_ms_min=0.200,
    think_ms_max=0.480,
    between_cpfs_min=1.8,
    between_cpfs_max=4.2,
    coffee_every=12,
    coffee_ms_min=18.0,
    coffee_ms_max=35.0,
    mistake_prob=0.00,
    enable_pointer_move=True,      # <-- ligar movimento
    click_noise_prob=0.58,         # mais variedade de clique em offsets
    page_noise_prob=0.25,          # às vezes clicar no vazio
    prefer_center_bias=0.70        # tende ao centro, mas não sempre
))

## Consulta de Dados

In [ ]:
try:

    for index, row in df.iterrows():
        try:
           # hb.random_page_click_noise()
            
            cpf_original = str(row['cpf']).split('.')[0].zfill(11)
            cpf_formatado = formatar_cpf(cpf_original)
           
            print(f"\nConsultando CPF: {cpf_formatado}...")
            
            
            
            def digitar_com_actions(driver, element, texto, min_delay=0.015, max_delay=0.045):
                actions = ActionChains(driver)
                actions.move_to_element(element).click()
                for ch in texto:
                    actions.send_keys(ch).pause(random.uniform(min_delay, max_delay))
                actions.perform()
            
            # --- Focar o campo de CPF com segurança (lida com o label sobreposto) ---
            
            campo_cpf = hb.safe_focus_input(
                (By.ID, 'document-input'),
                label_selector="label[for='document-input']",
                max_tries=3
            )
                       
            # Clique “variante” no próprio campo antes de digitar (ajuda a imitar humano)
            hb.click_like_human(campo_cpf)
                       
            hb.between_cpfs_pause(index)
            hb.clear_like_human(campo_cpf)
            hb.type_like_human(campo_cpf, cpf_formatado)
            hb._pause(hb.cfg.think_ms_min, hb.cfg.think_ms_max)
            
            # “Think time”
            hb._pause(hb.cfg.think_ms_min, hb.cfg.think_ms_max)
            
            # Garante absorção pela máscara
            if not hb.wait_field_value_digits(campo_cpf, cpf_formatado, timeout=5):
                hb.clear_like_human(campo_cpf)
                campo_cpf.send_keys(cpf_formatado)   # fallback simples e estável
                time.sleep(0.15)
            
            # Opcional: blur (algumas máscaras consolidam no TAB)
            campo_cpf.send_keys(Keys.TAB)
            time.sleep(0.06)
            
            # Clicar na “seta” de forma resiliente

            botao_consultar = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.sc-33edc40-3')))
            hb.click_like_human(botao_consultar)

            # --- NOVA LÓGICA DE VERIFICAÇÃO ---
            
           
            # --- AJUSTE AQUI: seletor da mensagem de erro no site ---
            # Exemplo: SELETOR_MENSAGEM_ERRO = (By.CSS_SELECTOR, 'p.sc-8042d590-2')
            SELETOR_MENSAGEM_ERRO = (By.CSS_SELECTOR, 'p.sc-12345-a')  # <--- AJUSTE ESTE SELETOR
            
            try:
                # Espera pelo PRIMEIRO que aparecer: botão de sucesso OU mensagem de erro
                wait.until(
                    EC.any_of(
                        EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Nova pesquisa')]")),
                        EC.presence_of_element_located(SELETOR_MENSAGEM_ERRO)
                    )
                )
            
                # Verifica se apareceu a mensagem de erro
                try:
                    mensagem_erro_site = driver.find_element(*SELETOR_MENSAGEM_ERRO).text
                    print(f"[⛔ERRO NO SITE] {mensagem_erro_site}")
                    set_result(index, f"Erro no site: {mensagem_erro_site}")
            
                    # Recarrega e segue pro próximo CPF
                    driver.get(url_negociacao)
                    wait.until(EC.element_to_be_clickable((By.ID, 'document-input')))
                    continue
                
                except NoSuchElementException:
                    # Não achou erro → segue (sucesso)
                    pass
                
            except TimeoutException:
                print(f"[⛔ERRO] Timeout no CPF {cpf_formatado}. Nem sucesso, nem erro encontrado.")
                resultados.append('Erro (Timeout na verificação)')
                driver.get(url_negociacao)  # Recarrega
                wait.until(EC.element_to_be_clickable((By.ID, 'document-input')))
                continue  # Próximo CPF
            
            
            # 4. Verifica o resultado (SUCESSO)
            # Pega TODAS as <li> de cobranças (ajuste se esse seletor mudar)
            dividas_list = driver.find_elements(By.CSS_SELECTOR, 'li.sc-35fcfe96-0')
            
            if len(dividas_list) > 0:
                meses = []
                detalhes = []  # para guardar (mes, valor, vencimento, status)
            
                for item in dividas_list:
                    try:
                        # Campos principais pela semântica (type=...)
                        mes_txt   = item.find_element(By.CSS_SELECTOR, 'div[type="month"]').text         # ex.: "Conta de Novembro"
                        valor_txt = item.find_element(By.CSS_SELECTOR, 'div[type="amountDue"]').text     # ex.: "R$ 100,00"
                        venc_txt  = item.find_element(By.CSS_SELECTOR, 'div[type="paymentDueDate"]').text  # ex.: "Vence dia 26/11/2025"
            
                        # Normalizações simples
                        mes = mes_txt.replace("Conta de ", "").strip()
                        vencimento = venc_txt.replace("Vence dia", "").strip()
                        valor = valor_txt.strip()
            
                        # ====== Status (com fallback robusto) ======
                        status = None
                        try:
                            # 1) Tenta por classe conhecida do card de status
                            status = item.find_element(By.CSS_SELECTOR, '.sc-76d7a608-0 p').text.strip()
                        except NoSuchElementException:
                            try:
                                # 2) Fallback: qualquer <p> na coluna da direita
                                status = item.find_element(
                                    By.XPATH,
                                    ".//div[contains(@class,'sc-35fcfe96-2')]//p"
                                ).text.strip()
                            except NoSuchElementException:
                                status = "Status indisponível"
                        # ==========================================
                        if mes:
                            meses.append(mes)
            
                        detalhes.append({
                            "mes": mes,
                            "valor": valor,
                            "vencimento": vencimento,
                            "status": status
                        })
            
                    except NoSuchElementException:
                        # Se algum item não tiver estrutura completa, ignora apenas este item
                        continue
                    
                # Remove duplicados de meses mantendo a ordem
                meses_unicos = list(dict.fromkeys(meses))
            
                # Monta o texto do resultado
                if len(meses_unicos) == 1 and len(detalhes) == 1:
                    d = detalhes[0]
                    resultado_final = (
                        f"Cobrança ativa do mês {d['mes']} | "
                        f"Valor: {d['valor']} | Vencimento: {d['vencimento']} | Status: {d['status']}"
                    )
                else:
                    # Vários meses: lista meses e exibe detalhes por item
                    detalhe_str = " || ".join(
                        [f"{d['mes']} • {d['valor']} • {d['vencimento']} • {d['status']}" for d in detalhes]
                    )
                    resultado_final = (
                        f"Cobranças ativas dos meses: {', '.join(meses_unicos)} | "
                        f"Detalhes: {detalhe_str}"
                    )
            
                print(f"[❌ RESULTADO ❌] {resultado_final}")
                set_result(index, resultado_final)
                gravou_sucesso = True
            
            else:
                # Nenhuma cobrança encontrada → tenta achar a mensagem de "sem contas"
                try:
                    # AJUSTE este seletor se necessário, pois também é classe gerada
                    mensagem_sem_divida = driver.find_element(By.CSS_SELECTOR, 'span.sc-8042d590-2').text
                    print(f"[✔ RESULTADO ✔] {mensagem_sem_divida}")
                    set_result(index, mensagem_sem_divida)
                    gravou_sucesso = True
                except NoSuchElementException:
                    print("[AVISO] Não encontrou dívidas nem mensagem de 'sem contas'.")
                    set_result(index, "Resultado inesperado (sucesso)")
                    gravou_sucesso = True
            
            # 5. Próxima consulta → apenas recarregar a página (SEM clicar em 'Nova pesquisa')
            driver.get(url_negociacao)
            wait.until(EC.element_to_be_clickable((By.ID, 'document-input')))
            
            # (Opcional) autosave a cada 10 linhas
            if (index + 1) % 10 == 0:
                df.to_excel('resultado_consulta_dividas.xlsx', index=False, engine='openpyxl')
                print("[SALVO] Progresso parcial em 'resultado_consulta_dividas.xlsx'.")
            
    
       
        except Exception as e:
            # Não sobrescrever o resultado se já gravou sucesso!
            print(f"[⛔ERRO] Falha inesperada no CPF {cpf_formatado}: {e}")
            if not gravou_sucesso:  # --- NOVO ---
                set_result(index, f"Erro na consulta: {type(e).__name__}: {e}")
            else:
                print("[INFO] Resultado já estava salvo; mantendo o status e ignorando o erro tardio.")

            try:
                driver.get(url_negociacao)
                wait.until(EC.element_to_be_clickable((By.ID, 'document-input')))
            except Exception as e_refresh:
                print(f"Falha crítica ao tentar recarregar a página: {e_refresh}")
                break
            


finally:
    driver.quit()  # se quiser encerrar
    # Salva o DataFrame com os resultados
    try:
        df.to_excel('resultado_consulta_dividas.xlsx', index=False, engine='openpyxl')
        print("\nConsulta finalizada. Os resultados foram salvos em 'resultado_consulta_dividas.xlsx'.")
        print(df[['cpf','Status da Dívida']])
    except Exception as e:
        print(f"\n[ERRO] Não foi possível salvar o arquivo de resultados: {e}")
